In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"America/New_York\"")

In [ ]:
hex_project_id = _hex_json.loads("\"0198e707-161f-7003-bc17-9a8d006d2b71\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Web scrape attempt\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [ ]:
!uv pip install playwright
!playwright install

Using Python 3.11.11 environment at: /home/hexuser/.cache/pypoetry/virtualenvs/python-kernel-OtKFaj5M-py3.11
Resolved 4 packages in 105ms                                         
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)----     0 B/43.79 MiB                     
⠙ Preparing packages... (0/2)---- 14.90 KiB/43.79 MiB                   
pyee       ------------------------------     0 B/15.36 KiB
⠙ Preparing packages... (0/2)---- 14.90 KiB/43.79 MiB                   
pyee       ------------------------------ 14.91 KiB/15.36 KiB
⠙ Preparing packages... (0/2)---- 14.90 KiB/43.79 MiB                   
pyee       ------------------------------ 14.91 KiB/15.36 KiB
⠙ Preparing packages... (0/2)---- 30.90 KiB/43.79 MiB                   
pyee       ------------------------------ 14.91 KiB/15.36 KiB
⠙ Preparing packages... (0/2)---- 46.90 KiB/43.79 MiB                   
pyee       ------------------------------ 14.91 KiB/15.3

In [ ]:
import asyncio, re, random
from pathlib import Path
import pandas as pd
from urllib.parse import urljoin
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

BASE = "https://www.trulieve.com"
DISPENSARIES_URL = f"{BASE}/dispensaries"
CATEGORY_URL = f"{BASE}/category/flower/whole-flower"
SUBCATEGORY   = "Whole Flower"
OUT_PREFIX    = "trulieve_FL_whole_flower"

PRICE_RE        = re.compile(r"\$\s*([0-9]+(?:\.[0-9]{2})?)")
SIZE_RE         = re.compile(r"\b(0\.5g|1g|2g|3\.5g|7g|10g|14g|28g)\b", re.I)
THC_SINGLE_RE   = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
THC_RANGE_RE    = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%[^0-9]+([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
SIZE_MAP = {"0.5g":0.5,"1g":1.0,"2g":2.0,"3.5g":3.5,"7g":7.0,"10g":10.0,"14g":14.0,"28g":28.0}
def grams_from_size(s): return SIZE_MAP.get((s or "").lower())

def looks_like_florida(href, text):
    t=(text or "").upper(); h=(href or "").lower()
    return (", FL" in t) or t.endswith(" FL") or " FL " in t or any(v in h for v in ("/florida","-fl-")) or h.endswith(("/fl","-fl"))

def product_slug(href):
    try: return href.split("/product/",1)[1].split("?",1)[0].split("#",1)[0].strip("/")
    except: return href or ""

async def extract_fl_store_links(page):
    await page.goto(DISPENSARIES_URL, wait_until="networkidle")
    anchors = await page.locator("a[href^='/dispensaries/']").all()
    out, seen = [], set()
    for a in anchors:
        href = await a.get_attribute("href"); txt = (await a.text_content() or "").strip()
        if href and "/dispensaries/" in href and href not in seen and looks_like_florida(href, txt):
            seen.add(href); out.append((" ".join(txt.split()), urljoin(BASE, href)))
    uniq, names = [], set()
    for name, url in out:
        if name not in names: names.add(name); uniq.append((name, url))
    return uniq

async def load_all(page):
    while True:
        await page.mouse.wheel(0, 40000); await page.wait_for_timeout(random.randint(800, 1400))
        btn = page.get_by_role("button", name=re.compile(r"Load More", re.I))
        if await btn.count() and await btn.first().is_visible():
            try: await btn.first().click(); await page.wait_for_timeout(random.randint(1000,1600)); continue
            except: pass
        break

async def extract_price_from_card(card):
    c = card.locator(".price, [class*='price'], :text('$'):not(:text('Add to Wishlist'))")
    try:
        n = min(await c.count(), 4); texts=[]
        for i in range(n):
            t = await c.nth(i).text_content()
            if t and "$" in t: texts.append(t)
        blob = " ".join(texts) if texts else (await card.inner_text())
    except: blob = (await card.inner_text())
    vals = [float(m.group(1)) for m in PRICE_RE.finditer(blob or "")]
    return min(vals) if vals else None

async def extract_price_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        body = await p.locator("body").inner_text(); await p.close()
        vals = [float(m.group(1)) for m in PRICE_RE.finditer(body or "")]
        return min(vals) if vals else None
    except (PWTimeout, Exception): return None

async def extract_brand_from_card(card):
    try:
        bel = card.locator(".ProductCard_brand, .brand, .c-product-card__brand, [class*='Brand'], [data-testid*='brand']")
        if await bel.count():
            txt = (await bel.first.text_content() or "").strip()
            if txt: return txt
    except: pass
    return None

async def extract_brand_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        # breadcrumb / meta
        crumbs = p.locator("nav a, .breadcrumb a, [class*='breadcrumb'] a")
        if await crumbs.count():
            vals=[]
            for i in range(min(5, await crumbs.count())):
                t=(await crumbs.nth(i).text_content() or "").strip()
                if t: vals.append(t)
            for v in vals:
                if v.lower() not in ("home","flower","pre-rolls","minis","ground & shake","products","shop"):
                    if 1<=len(v)<=40: await p.close(); return v
        # labeled line
        label = p.locator("text=/Brand\\s*:/i")
        if await label.count():
            line = await label.first.text_content()
            if line and ":" in line:
                b=line.split(":",1)[1].strip()
                if b: await p.close(); return b
        meta = p.locator("[data-brand], [itemprop='brand'], [class*='brand']")
        if await meta.count():
            t=(await meta.first.text_content() or "").strip()
            if t: await p.close(); return t
        body = await p.locator("body").inner_text()
        m = re.search(r"Brand\s*[:\-]\s*([^\n\r]+)", body, flags=re.I)
        await p.close()
        return m.group(1).strip() if m else None
    except (PWTimeout, Exception): return None

async def scrape_category(page, ctx, store_name):
    await page.goto(CATEGORY_URL, wait_until="domcontentloaded"); await load_all(page)
    name_links = await page.locator("a[href*='/product/']:not(:has(img))").all()
    rows, seen = [], set()
    for link in name_links:
        try:
            name = ((await link.text_content()) or "").strip()
            if not name: continue
            href = await link.get_attribute("href"); url = urljoin(BASE, href) if href else None
            slug = product_slug(href or "")
            card = link.locator("xpath=ancestor::*[self::article or self::li or self::div][1]")
            card_text = (await card.inner_text()) if await card.count() else name

            size = (SIZE_RE.search(card_text).group(1).lower() if SIZE_RE.search(card_text) else None)
            grams = grams_from_size(size)

            price = await extract_price_from_card(card)
            brand = await extract_brand_from_card(card)
            if (price is None or brand is None) and url:
                if price is None: 
                    p = await extract_price_from_pdp(ctx, url)
                    if p is not None: price = p
                if brand is None:
                    b = await extract_brand_from_pdp(ctx, url)
                    if b: brand = b

            strain_type=None
            st_el = card.locator("text=/\\b(Indica|Sativa|Hybrid)\\b/i")
            if await st_el.count():
                t=(await st_el.first.text_content()) or ""
                m=re.search(r"(Indica|Sativa|Hybrid)", t, re.I)
                if m: strain_type=m.group(1).capitalize()
            if not strain_type:
                for kw in ("Indica","Sativa","Hybrid"):
                    if re.search(rf"\b{kw}\b", card_text, re.I): strain_type=kw; break

            thc_pct=None
            mr=THC_RANGE_RE.search(card_text)
            if mr:
                try: thc_pct=float(mr.group(1))
                except: pass
            if thc_pct is None:
                ms=THC_SINGLE_RE.search(card_text)
                if ms:
                    try: thc_pct=float(ms.group(1))
                    except: pass

            key=(store_name, slug, size, SUBCATEGORY)
            if key in seen: continue
            seen.add(key)

            rows.append({
                "state":"FL","store":store_name,"subcategory":SUBCATEGORY,"name":name,"brand":brand,
                "strain_type":strain_type,"thc_pct":thc_pct,"size_raw":size,"grams":grams,
                "price":price,"price_per_g":(round(price/grams,2) if price and grams else None),"url":url
            })
        except: continue
    return rows

async def run(headless=True):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, args=["--no-sandbox"])
        ctx = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126 Safari/537.36")
        page = await ctx.new_page()

        stores = await extract_fl_store_links(page)
        all_rows=[]
        for store_name, store_url in stores:
            await page.wait_for_timeout(random.randint(700,1500))
            try:
                await page.goto(store_url, wait_until="domcontentloaded")
                btn = page.get_by_role("button", name=re.compile(r"Shop At This Store", re.I))
                if await btn.count():
                    try: await btn.first().click(); await page.wait_for_timeout(random.randint(900,1400))
                    except: pass
            except: continue
            rows = await scrape_category(page, ctx, store_name); all_rows.extend(rows)

        await browser.close()

    global DF_WHOLE_FLOWER
    DF_WHOLE_FLOWER = pd.DataFrame(all_rows).sort_values(["store","brand","name","grams"], kind="stable")
    DF_WHOLE_FLOWER.to_csv(Path(f"{OUT_PREFIX}.csv"), index=False)
    DF_WHOLE_FLOWER.to_parquet(Path(f"{OUT_PREFIX}.parquet"), index=False)
    print(f"Wrote {len(DF_WHOLE_FLOWER):,} rows across {DF_WHOLE_FLOWER['store'].nunique() if len(DF_WHOLE_FLOWER) else 0} FL stores.")

if __name__ == "__main__":
    import sys
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    try:
        asyncio.get_running_loop()
        import nest_asyncio; nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(run(headless=True))
    except RuntimeError:
        asyncio.run(run(headless=True))


Wrote 2,544 rows across 159 FL stores.


In [ ]:
import asyncio, re, random
from pathlib import Path
import pandas as pd
from urllib.parse import urljoin
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

BASE = "https://www.trulieve.com"
DISPENSARIES_URL = f"{BASE}/dispensaries"
CATEGORY_URL = f"{BASE}/category/flower/minis"
SUBCATEGORY  = "Ground & Shake"
OUT_PREFIX   = "trulieve_FL_ground_shake"

PRICE_RE        = re.compile(r"\$\s*([0-9]+(?:\.[0-9]{2})?)")
SIZE_RE         = re.compile(r"\b(0\.5g|1g|2g|3\.5g|7g|10g|14g|28g)\b", re.I)
THC_SINGLE_RE   = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
THC_RANGE_RE    = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%[^0-9]+([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
SIZE_MAP = {"0.5g":0.5,"1g":1.0,"2g":2.0,"3.5g":3.5,"7g":7.0,"10g":10.0,"14g":14.0,"28g":28.0}
def grams_from_size(s): return SIZE_MAP.get((s or "").lower())

def looks_like_florida(href, text):
    t=(text or "").upper(); h=(href or "").lower()
    return (", FL" in t) or t.endswith(" FL") or " FL " in t or any(v in h for v in ("/florida","-fl-")) or h.endswith(("/fl","-fl"))

def product_slug(href):
    try: return href.split("/product/",1)[1].split("?",1)[0].split("#",1)[0].strip("/")
    except: return href or ""

async def extract_fl_store_links(page):
    await page.goto(DISPENSARIES_URL, wait_until="networkidle")
    anchors = await page.locator("a[href^='/dispensaries/']").all()
    out, seen = [], set()
    for a in anchors:
        href = await a.get_attribute("href"); txt = (await a.text_content() or "").strip()
        if href and "/dispensaries/" in href and href not in seen and looks_like_florida(href, txt):
            seen.add(href); out.append((" ".join(txt.split()), urljoin(BASE, href)))
    uniq, names = [], set()
    for name, url in out:
        if name not in names: names.add(name); uniq.append((name, url))
    return uniq

async def load_all(page):
    while True:
        await page.mouse.wheel(0, 40000); await page.wait_for_timeout(random.randint(800, 1400))
        btn = page.get_by_role("button", name=re.compile(r"Load More", re.I))
        if await btn.count() and await btn.first().is_visible():
            try: await btn.first().click(); await page.wait_for_timeout(random.randint(1000,1600)); continue
            except: pass
        break

async def extract_price_from_card(card):
    c = card.locator(".price, [class*='price'], :text('$'):not(:text('Add to Wishlist'))")
    try:
        n = min(await c.count(), 4); texts=[]
        for i in range(n):
            t = await c.nth(i).text_content()
            if t and "$" in t: texts.append(t)
        blob = " ".join(texts) if texts else (await card.inner_text())
    except: blob = (await card.inner_text())
    vals = [float(m.group(1)) for m in PRICE_RE.finditer(blob or "")]
    return min(vals) if vals else None

async def extract_price_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        body = await p.locator("body").inner_text(); await p.close()
        vals = [float(m.group(1)) for m in PRICE_RE.finditer(body or "")]
        return min(vals) if vals else None
    except (PWTimeout, Exception): return None

async def extract_brand_from_card(card):
    try:
        bel = card.locator(".ProductCard_brand, .brand, .c-product-card__brand, [class*='Brand'], [data-testid*='brand']")
        if await bel.count():
            txt = (await bel.first.text_content() or "").strip()
            if txt: return txt
    except: pass
    return None

async def extract_brand_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        # breadcrumb / meta
        crumbs = p.locator("nav a, .breadcrumb a, [class*='breadcrumb'] a")
        if await crumbs.count():
            vals=[]
            for i in range(min(5, await crumbs.count())):
                t=(await crumbs.nth(i).text_content() or "").strip()
                if t: vals.append(t)
            for v in vals:
                if v.lower() not in ("home","flower","pre-rolls","minis","ground & shake","products","shop"):
                    if 1<=len(v)<=40: await p.close(); return v
        # labeled line
        label = p.locator("text=/Brand\\s*:/i")
        if await label.count():
            line = await label.first.text_content()
            if line and ":" in line:
                b=line.split(":",1)[1].strip()
                if b: await p.close(); return b
        meta = p.locator("[data-brand], [itemprop='brand'], [class*='brand']")
        if await meta.count():
            t=(await meta.first.text_content() or "").strip()
            if t: await p.close(); return t
        body = await p.locator("body").inner_text()
        m = re.search(r"Brand\s*[:\-]\s*([^\n\r]+)", body, flags=re.I)
        await p.close()
        return m.group(1).strip() if m else None
    except (PWTimeout, Exception): return None

async def scrape_category(page, ctx, store_name):
    await page.goto(CATEGORY_URL, wait_until="domcontentloaded"); await load_all(page)
    name_links = await page.locator("a[href*='/product/']:not(:has(img))").all()
    rows, seen = [], set()
    for link in name_links:
        try:
            name = ((await link.text_content()) or "").strip()
            if not name: continue
            href = await link.get_attribute("href"); url = urljoin(BASE, href) if href else None
            slug = product_slug(href or "")
            card = link.locator("xpath=ancestor::*[self::article or self::li or self::div][1]")
            card_text = (await card.inner_text()) if await card.count() else name

            size = (SIZE_RE.search(card_text).group(1).lower() if SIZE_RE.search(card_text) else None)
            grams = grams_from_size(size)

            price = await extract_price_from_card(card)
            brand = await extract_brand_from_card(card)
            if (price is None or brand is None) and url:
                if price is None: 
                    p = await extract_price_from_pdp(ctx, url)
                    if p is not None: price = p
                if brand is None:
                    b = await extract_brand_from_pdp(ctx, url)
                    if b: brand = b

            strain_type=None
            st_el = card.locator("text=/\\b(Indica|Sativa|Hybrid)\\b/i")
            if await st_el.count():
                t=(await st_el.first.text_content()) or ""
                m=re.search(r"(Indica|Sativa|Hybrid)", t, re.I)
                if m: strain_type=m.group(1).capitalize()
            if not strain_type:
                for kw in ("Indica","Sativa","Hybrid"):
                    if re.search(rf"\b{kw}\b", card_text, re.I): strain_type=kw; break

            thc_pct=None
            mr=THC_RANGE_RE.search(card_text)
            if mr:
                try: thc_pct=float(mr.group(1))
                except: pass
            if thc_pct is None:
                ms=THC_SINGLE_RE.search(card_text)
                if ms:
                    try: thc_pct=float(ms.group(1))
                    except: pass

            key=(store_name, slug, size, SUBCATEGORY)
            if key in seen: continue
            seen.add(key)

            rows.append({
                "state":"FL","store":store_name,"subcategory":SUBCATEGORY,"name":name,"brand":brand,
                "strain_type":strain_type,"thc_pct":thc_pct,"size_raw":size,"grams":grams,
                "price":price,"price_per_g":(round(price/grams,2) if price and grams else None),"url":url
            })
        except: continue
    return rows

async def run(headless=True):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, args=["--no-sandbox"])
        ctx = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126 Safari/537.36")
        page = await ctx.new_page()

        stores = await extract_fl_store_links(page)
        all_rows=[]
        for store_name, store_url in stores:
            await page.wait_for_timeout(random.randint(700,1500))
            try:
                await page.goto(store_url, wait_until="domcontentloaded")
                btn = page.get_by_role("button", name=re.compile(r"Shop At This Store", re.I))
                if await btn.count():
                    try: await btn.first().click(); await page.wait_for_timeout(random.randint(900,1400))
                    except: pass
            except: continue
            rows = await scrape_category(page, ctx, store_name); all_rows.extend(rows)

        await browser.close()

    global DF_GROUND_SHAKE
    DF_GROUND_SHAKE = pd.DataFrame(all_rows).sort_values(["store","brand","name","grams"], kind="stable")
    DF_GROUND_SHAKE.to_csv(Path(f"{OUT_PREFIX}.csv"), index=False)
    DF_GROUND_SHAKE.to_parquet(Path(f"{OUT_PREFIX}.parquet"), index=False)
    print(f"Wrote {len(DF_GROUND_SHAKE):,} rows across {DF_GROUND_SHAKE['store'].nunique() if len(DF_GROUND_SHAKE) else 0} FL stores.")

if __name__ == "__main__":
    import sys
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    try:
        asyncio.get_running_loop()
        import nest_asyncio; nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(run(headless=True))
    except RuntimeError:
        asyncio.run(run(headless=True))


Wrote 2,544 rows across 159 FL stores.


In [ ]:
import asyncio, re, random
from pathlib import Path
import pandas as pd
from urllib.parse import urljoin
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

BASE = "https://www.trulieve.com"
DISPENSARIES_URL = f"{BASE}/dispensaries"
CATEGORY_URL = f"{BASE}/category/flower/pre-rolls"
SUBCATEGORY  = "Pre-Rolls"
OUT_PREFIX   = "trulieve_FL_pre_rolls"

PRICE_RE        = re.compile(r"\$\s*([0-9]+(?:\.[0-9]{2})?)")
SIZE_RE         = re.compile(r"\b(0\.5g|1g|2g|3\.5g|7g|10g|14g|28g)\b", re.I)
THC_SINGLE_RE   = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
THC_RANGE_RE    = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%[^0-9]+([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
SIZE_MAP = {"0.5g":0.5,"1g":1.0,"2g":2.0,"3.5g":3.5,"7g":7.0,"10g":10.0,"14g":14.0,"28g":28.0}
def grams_from_size(s): return SIZE_MAP.get((s or "").lower())

def looks_like_florida(href, text):
    t=(text or "").upper(); h=(href or "").lower()
    return (", FL" in t) or t.endswith(" FL") or " FL " in t or any(v in h for v in ("/florida","-fl-")) or h.endswith(("/fl","-fl"))

def product_slug(href):
    try: return href.split("/product/",1)[1].split("?",1)[0].split("#",1)[0].strip("/")
    except: return href or ""

async def extract_fl_store_links(page):
    await page.goto(DISPENSARIES_URL, wait_until="networkidle")
    anchors = await page.locator("a[href^='/dispensaries/']").all()
    out, seen = [], set()
    for a in anchors:
        href = await a.get_attribute("href"); txt = (await a.text_content() or "").strip()
        if href and "/dispensaries/" in href and href not in seen and looks_like_florida(href, txt):
            seen.add(href); out.append((" ".join(txt.split()), urljoin(BASE, href)))
    uniq, names = [], set()
    for name, url in out:
        if name not in names: names.add(name); uniq.append((name, url))
    return uniq

async def load_all(page):
    while True:
        await page.mouse.wheel(0, 40000); await page.wait_for_timeout(random.randint(800, 1400))
        btn = page.get_by_role("button", name=re.compile(r"Load More", re.I))
        if await btn.count() and await btn.first().is_visible():
            try: await btn.first().click(); await page.wait_for_timeout(random.randint(1000,1600)); continue
            except: pass
        break

async def extract_price_from_card(card):
    c = card.locator(".price, [class*='price'], :text('$'):not(:text('Add to Wishlist'))")
    try:
        n = min(await c.count(), 4); texts=[]
        for i in range(n):
            t = await c.nth(i).text_content()
            if t and "$" in t: texts.append(t)
        blob = " ".join(texts) if texts else (await card.inner_text())
    except: blob = (await card.inner_text())
    vals = [float(m.group(1)) for m in PRICE_RE.finditer(blob or "")]
    return min(vals) if vals else None

async def extract_price_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        body = await p.locator("body").inner_text(); await p.close()
        vals = [float(m.group(1)) for m in PRICE_RE.finditer(body or "")]
        return min(vals) if vals else None
    except (PWTimeout, Exception): return None

async def extract_brand_from_card(card):
    try:
        bel = card.locator(".ProductCard_brand, .brand, .c-product-card__brand, [class*='Brand'], [data-testid*='brand']")
        if await bel.count():
            txt = (await bel.first.text_content() or "").strip()
            if txt: return txt
    except: pass
    return None

async def extract_brand_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        # breadcrumb / meta
        crumbs = p.locator("nav a, .breadcrumb a, [class*='breadcrumb'] a")
        if await crumbs.count():
            vals=[]
            for i in range(min(5, await crumbs.count())):
                t=(await crumbs.nth(i).text_content() or "").strip()
                if t: vals.append(t)
            for v in vals:
                if v.lower() not in ("home","flower","pre-rolls","minis","ground & shake","products","shop"):
                    if 1<=len(v)<=40: await p.close(); return v
        # labeled line
        label = p.locator("text=/Brand\\s*:/i")
        if await label.count():
            line = await label.first.text_content()
            if line and ":" in line:
                b=line.split(":",1)[1].strip()
                if b: await p.close(); return b
        meta = p.locator("[data-brand], [itemprop='brand'], [class*='brand']")
        if await meta.count():
            t=(await meta.first.text_content() or "").strip()
            if t: await p.close(); return t
        body = await p.locator("body").inner_text()
        m = re.search(r"Brand\s*[:\-]\s*([^\n\r]+)", body, flags=re.I)
        await p.close()
        return m.group(1).strip() if m else None
    except (PWTimeout, Exception): return None

async def scrape_category(page, ctx, store_name):
    await page.goto(CATEGORY_URL, wait_until="domcontentloaded"); await load_all(page)
    name_links = await page.locator("a[href*='/product/']:not(:has(img))").all()
    rows, seen = [], set()
    for link in name_links:
        try:
            name = ((await link.text_content()) or "").strip()
            if not name: continue
            href = await link.get_attribute("href"); url = urljoin(BASE, href) if href else None
            slug = product_slug(href or "")
            card = link.locator("xpath=ancestor::*[self::article or self::li or self::div][1]")
            card_text = (await card.inner_text()) if await card.count() else name

            size = (SIZE_RE.search(card_text).group(1).lower() if SIZE_RE.search(card_text) else None)
            grams = grams_from_size(size)

            price = await extract_price_from_card(card)
            brand = await extract_brand_from_card(card)
            if (price is None or brand is None) and url:
                if price is None: 
                    p = await extract_price_from_pdp(ctx, url)
                    if p is not None: price = p
                if brand is None:
                    b = await extract_brand_from_pdp(ctx, url)
                    if b: brand = b

            strain_type=None
            st_el = card.locator("text=/\\b(Indica|Sativa|Hybrid)\\b/i")
            if await st_el.count():
                t=(await st_el.first.text_content()) or ""
                m=re.search(r"(Indica|Sativa|Hybrid)", t, re.I)
                if m: strain_type=m.group(1).capitalize()
            if not strain_type:
                for kw in ("Indica","Sativa","Hybrid"):
                    if re.search(rf"\b{kw}\b", card_text, re.I): strain_type=kw; break

            thc_pct=None
            mr=THC_RANGE_RE.search(card_text)
            if mr:
                try: thc_pct=float(mr.group(1))
                except: pass
            if thc_pct is None:
                ms=THC_SINGLE_RE.search(card_text)
                if ms:
                    try: thc_pct=float(ms.group(1))
                    except: pass

            key=(store_name, slug, size, SUBCATEGORY)
            if key in seen: continue
            seen.add(key)

            rows.append({
                "state":"FL","store":store_name,"subcategory":SUBCATEGORY,"name":name,"brand":brand,
                "strain_type":strain_type,"thc_pct":thc_pct,"size_raw":size,"grams":grams,
                "price":price,"price_per_g":(round(price/grams,2) if price and grams else None),"url":url
            })
        except: continue
    return rows

async def run(headless=True):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, args=["--no-sandbox"])
        ctx = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126 Safari/537.36")
        page = await ctx.new_page()

        stores = await extract_fl_store_links(page)
        all_rows=[]
        for store_name, store_url in stores:
            await page.wait_for_timeout(random.randint(700,1500))
            try:
                await page.goto(store_url, wait_until="domcontentloaded")
                btn = page.get_by_role("button", name=re.compile(r"Shop At This Store", re.I))
                if await btn.count():
                    try: await btn.first().click(); await page.wait_for_timeout(random.randint(900,1400))
                    except: pass
            except: continue
            rows = await scrape_category(page, ctx, store_name); all_rows.extend(rows)

        await browser.close()

        global DF_PRE_ROLLS
        DF_PRE_ROLLS = pd.DataFrame(all_rows).sort_values(["store","brand","name","grams"], kind="stable")
        DF_PRE_ROLLS.to_csv(Path(f"{OUT_PREFIX}.csv"), index=False)
        DF_PRE_ROLLS.to_parquet(Path(f"{OUT_PREFIX}.parquet"), index=False)
        print(f"Wrote {len(DF_PRE_ROLLS):,} rows across {DF_PRE_ROLLS['store'].nunique() if len(DF_PRE_ROLLS) else 0} FL stores.")

if __name__ == "__main__":
    import sys
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    try:
        asyncio.get_running_loop()
        import nest_asyncio; nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(run(headless=True))
    except RuntimeError:
        asyncio.run(run(headless=True))


Wrote 2,544 rows across 159 FL stores.


In [ ]:
import asyncio, re, random
from pathlib import Path
import pandas as pd
from urllib.parse import urljoin
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

BASE = "https://www.trulieve.com"
DISPENSARIES_URL = f"{BASE}/dispensaries"
CATEGORY_URL = f"{BASE}/category/flower/ground-shake"
SUBCATEGORY  = "Ground & Shake"
OUT_PREFIX   = "trulieve_FL_ground_shake"

PRICE_RE        = re.compile(r"\$\s*([0-9]+(?:\.[0-9]{2})?)")
SIZE_RE         = re.compile(r"\b(0\.5g|1g|2g|3\.5g|7g|10g|14g|28g)\b", re.I)
THC_SINGLE_RE   = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
THC_RANGE_RE    = re.compile(r"\bTHC\b[^0-9]*([0-9]+(?:\.[0-9]+)?)\s*%[^0-9]+([0-9]+(?:\.[0-9]+)?)\s*%", re.I)
SIZE_MAP = {"0.5g":0.5,"1g":1.0,"2g":2.0,"3.5g":3.5,"7g":7.0,"10g":10.0,"14g":14.0,"28g":28.0}
def grams_from_size(s): return SIZE_MAP.get((s or "").lower())

def looks_like_florida(href, text):
    t=(text or "").upper(); h=(href or "").lower()
    return (", FL" in t) or t.endswith(" FL") or " FL " in t or any(v in h for v in ("/florida","-fl-")) or h.endswith(("/fl","-fl"))

def product_slug(href):
    try: return href.split("/product/",1)[1].split("?",1)[0].split("#",1)[0].strip("/")
    except: return href or ""

async def extract_fl_store_links(page):
    await page.goto(DISPENSARIES_URL, wait_until="networkidle")
    anchors = await page.locator("a[href^='/dispensaries/']").all()
    out, seen = [], set()
    for a in anchors:
        href = await a.get_attribute("href"); txt = (await a.text_content() or "").strip()
        if href and "/dispensaries/" in href and href not in seen and looks_like_florida(href, txt):
            seen.add(href); out.append((" ".join(txt.split()), urljoin(BASE, href)))
    uniq, names = [], set()
    for name, url in out:
        if name not in names: names.add(name); uniq.append((name, url))
    return uniq

async def load_all(page):
    while True:
        await page.mouse.wheel(0, 40000); await page.wait_for_timeout(random.randint(800, 1400))
        btn = page.get_by_role("button", name=re.compile(r"Load More", re.I))
        if await btn.count() and await btn.first().is_visible():
            try: await btn.first().click(); await page.wait_for_timeout(random.randint(1000,1600)); continue
            except: pass
        break

async def extract_price_from_card(card):
    c = card.locator(".price, [class*='price'], :text('$'):not(:text('Add to Wishlist'))")
    try:
        n = min(await c.count(), 4); texts=[]
        for i in range(n):
            t = await c.nth(i).text_content()
            if t and "$" in t: texts.append(t)
        blob = " ".join(texts) if texts else (await card.inner_text())
    except: blob = (await card.inner_text())
    vals = [float(m.group(1)) for m in PRICE_RE.finditer(blob or "")]
    return min(vals) if vals else None

async def extract_price_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        body = await p.locator("body").inner_text(); await p.close()
        vals = [float(m.group(1)) for m in PRICE_RE.finditer(body or "")]
        return min(vals) if vals else None
    except (PWTimeout, Exception): return None

async def extract_brand_from_card(card):
    try:
        bel = card.locator(".ProductCard_brand, .brand, .c-product-card__brand, [class*='Brand'], [data-testid*='brand']")
        if await bel.count():
            txt = (await bel.first.text_content() or "").strip()
            if txt: return txt
    except: pass
    return None

async def extract_brand_from_pdp(ctx, url):
    try:
        p = await ctx.new_page()
        await p.goto(url, wait_until="domcontentloaded", timeout=20000)
        # breadcrumb / meta
        crumbs = p.locator("nav a, .breadcrumb a, [class*='breadcrumb'] a")
        if await crumbs.count():
            vals=[]
            for i in range(min(5, await crumbs.count())):
                t=(await crumbs.nth(i).text_content() or "").strip()
                if t: vals.append(t)
            for v in vals:
                if v.lower() not in ("home","flower","pre-rolls","minis","ground & shake","products","shop"):
                    if 1<=len(v)<=40: await p.close(); return v
        # labeled line
        label = p.locator("text=/Brand\\s*:/i")
        if await label.count():
            line = await label.first.text_content()
            if line and ":" in line:
                b=line.split(":",1)[1].strip()
                if b: await p.close(); return b
        meta = p.locator("[data-brand], [itemprop='brand'], [class*='brand']")
        if await meta.count():
            t=(await meta.first.text_content() or "").strip()
            if t: await p.close(); return t
        body = await p.locator("body").inner_text()
        m = re.search(r"Brand\s*[:\-]\s*([^\n\r]+)", body, flags=re.I)
        await p.close()
        return m.group(1).strip() if m else None
    except (PWTimeout, Exception): return None

async def scrape_category(page, ctx, store_name):
    await page.goto(CATEGORY_URL, wait_until="domcontentloaded"); await load_all(page)
    name_links = await page.locator("a[href*='/product/']:not(:has(img))").all()
    rows, seen = [], set()
    for link in name_links:
        try:
            name = ((await link.text_content()) or "").strip()
            if not name: continue
            href = await link.get_attribute("href"); url = urljoin(BASE, href) if href else None
            slug = product_slug(href or "")
            card = link.locator("xpath=ancestor::*[self::article or self::li or self::div][1]")
            card_text = (await card.inner_text()) if await card.count() else name

            size = (SIZE_RE.search(card_text).group(1).lower() if SIZE_RE.search(card_text) else None)
            grams = grams_from_size(size)

            price = await extract_price_from_card(card)
            brand = await extract_brand_from_card(card)
            if (price is None or brand is None) and url:
                if price is None: 
                    p = await extract_price_from_pdp(ctx, url)
                    if p is not None: price = p
                if brand is None:
                    b = await extract_brand_from_pdp(ctx, url)
                    if b: brand = b

            strain_type=None
            st_el = card.locator("text=/\\b(Indica|Sativa|Hybrid)\\b/i")
            if await st_el.count():
                t=(await st_el.first.text_content()) or ""
                m=re.search(r"(Indica|Sativa|Hybrid)", t, re.I)
                if m: strain_type=m.group(1).capitalize()
            if not strain_type:
                for kw in ("Indica","Sativa","Hybrid"):
                    if re.search(rf"\b{kw}\b", card_text, re.I): strain_type=kw; break

            thc_pct=None
            mr=THC_RANGE_RE.search(card_text)
            if mr:
                try: thc_pct=float(mr.group(1))
                except: pass
            if thc_pct is None:
                ms=THC_SINGLE_RE.search(card_text)
                if ms:
                    try: thc_pct=float(ms.group(1))
                    except: pass

            key=(store_name, slug, size, SUBCATEGORY)
            if key in seen: continue
            seen.add(key)

            rows.append({
                "state":"FL","store":store_name,"subcategory":SUBCATEGORY,"name":name,"brand":brand,
                "strain_type":strain_type,"thc_pct":thc_pct,"size_raw":size,"grams":grams,
                "price":price,"price_per_g":(round(price/grams,2) if price and grams else None),"url":url
            })
        except: continue
    return rows

async def run(headless=True):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, args=["--no-sandbox"])
        ctx = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126 Safari/537.36")
        page = await ctx.new_page()

        stores = await extract_fl_store_links(page)
        all_rows=[]
        for store_name, store_url in stores:
            await page.wait_for_timeout(random.randint(700,1500))
            try:
                await page.goto(store_url, wait_until="domcontentloaded")
                btn = page.get_by_role("button", name=re.compile(r"Shop At This Store", re.I))
                if await btn.count():
                    try: await btn.first().click(); await page.wait_for_timeout(random.randint(900,1400))
                    except: pass
            except: continue
            rows = await scrape_category(page, ctx, store_name); all_rows.extend(rows)

        await browser.close()
        global DF_PRE_ROLLS
        DF_PRE_ROLLS = pd.DataFrame(all_rows).sort_values(["store","brand","name","grams"], kind="stable")
        DF_PRE_ROLLS.to_csv(Path(f"{OUT_PREFIX}.csv"), index=False)
        DF_PRE_ROLLS.to_parquet(Path(f"{OUT_PREFIX}.parquet"), index=False)
        print(f"Wrote {len(DF_PRE_ROLLS):,} rows across {DF_PRE_ROLLS['store'].nunique() if len(DF_PRE_ROLLS) else 0} FL stores.")
        

if __name__ == "__main__":
    import sys
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    try:
        asyncio.get_running_loop()
        import nest_asyncio; nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(run(headless=True))
    except RuntimeError:
        asyncio.run(run(headless=True))


Wrote 2,544 rows across 159 FL stores.


In [ ]:
# if hex_run_context in ["logic"]:
#     import hextoolkit
#     hex_data_connection = hextoolkit.get_data_connection("Curaleaf Production Snowflake (New)")
#     writeback_metadata = hex_data_connection.write_dataframe(df=DF_WHOLE_FLOWER, database="SANDBOX_EDW", schema="ANALYTICS", table="TL_Scrape_WHOLE_FLOWER", overwrite=False)

In [ ]:
# if hex_run_context in ["logic"]:
#     import hextoolkit
#     hex_data_connection = hextoolkit.get_data_connection("Curaleaf Production Snowflake (New)")
#     writeback_metadata = hex_data_connection.write_dataframe(df=DF_PRE_ROLLS, database="SANDBOX_EDW", schema="ANALYTICS", table="TL_Scrape_Pre_Rolls", overwrite=False)

In [ ]:
# if hex_run_context in ["logic"]:
#     import hextoolkit
#     hex_data_connection = hextoolkit.get_data_connection("Curaleaf Production Snowflake (New)")
#     writeback_metadata = hex_data_connection.write_dataframe(df=DF_GROUND_SHAKE, database="SANDBOX_EDW", schema="ANALYTICS", table="TL_Scrape_Ground_Shake", overwrite=False)

To do:

- add time stamp to generated DF
- publish and automate
- Solve Brand miss
- replicate for Muv

